In [1]:
import os
# Find the latest version of spark 3.0 from http://www.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.0.3'
spark_version = 'spark-3.0.3'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop2.7.tgz
!tar xf $SPARK_VERSION-bin-hadoop2.7.tgz
!pip install -q findspark

# Set Environment Variables
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()

Get:1 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Ign:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Get:3 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease [15.9 kB]
Get:4 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Ign:5 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:6 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release
Hit:7 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Hit:8 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:9 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Hit:10 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Hit:11 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease
Get:12 http://security.ubuntu.com/ubuntu bionic-security/main amd64 Packages [2,372 kB]
Get:13

In [2]:
# Download the Postgres driver that will allow Spark to interact with Postgres.
!wget https://jdbc.postgresql.org/download/postgresql-42.2.16.jar

--2021-10-19 22:14:56--  https://jdbc.postgresql.org/download/postgresql-42.2.16.jar
Resolving jdbc.postgresql.org (jdbc.postgresql.org)... 72.32.157.228, 2001:4800:3e1:1::228
Connecting to jdbc.postgresql.org (jdbc.postgresql.org)|72.32.157.228|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1002883 (979K) [application/java-archive]
Saving to: ‘postgresql-42.2.16.jar’

postgresql-42.2.16. 100%[===================>] 979.38K  --.-KB/s    in 0.1s    

2021-10-19 22:14:56 (6.50 MB/s) - ‘postgresql-42.2.16.jar’ saved [1002883/1002883]



In [3]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("M16-Amazon-Challenge").config("spark.driver.extraClassPath","/content/postgresql-42.2.16.jar").getOrCreate()

**Load Amazon Data into Spark DataFrame**

In [4]:
from pyspark import SparkFiles
url = "https://s3.amazonaws.com/amazon-reviews-pds/tsv/amazon_reviews_us_Sports_v1_00.tsv.gz"
spark.sparkContext.addFile(url)
df = spark.read.option("encoding", "UTF-8").csv(SparkFiles.get(""), sep="\t", header=True, inferSchema=True)
df.show()

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|   48945260|R1WBPB8MDCCN8F|B012P7UPSM|     409940130|Chicago Blackhawk...|          Sports|          5|            0|          0|   N|                N|   LOVE IT. 6 stars!|Bought this last ...| 2015-08-31|
|         US|    5782091|R32M0YEWV77XG8|B001GQ3VHG|     657746679|Copag Poker Size ...|          Sports|          5|    

**Create DataFrames to Match Tables**

In [5]:
# Create vine_table DataFrame
vine_df = df.select(["review_id", "star_rating", "helpful_votes", "total_votes", "vine", "verified_purchase"])
vine_df.show()

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
|R1WBPB8MDCCN8F|          5|            0|          0|   N|                N|
|R32M0YEWV77XG8|          5|            1|          1|   N|                Y|
| RR8V7WR27NXJ5|          1|            0|          0|   N|                Y|
|R1MHO5V9Z932AY|          5|            0|          0|   N|                Y|
|R16PD71086BD2V|          5|            0|          1|   N|                N|
|R1Z8IFGWTRWXT6|          3|            0|          0|   N|                Y|
|R3AUMSHAW73HWN|          4|            2|          3|   N|                N|
|R2KWDWFOHGX6FL|          5|            0|          0|   N|                Y|
|R3H9543FWBWFBU|          5|            1|          1|   N|                Y|
| RUANXOQ9W3OU5|          5|            0|          0|   N|     

In [7]:
# Convert star_rating column to integer
from pyspark.sql.types import IntegerType
vine_df = vine_df.withColumn("star_rating",vine_df.star_rating.cast(IntegerType()))
vine_df.dtypes

[('review_id', 'string'),
 ('star_rating', 'int'),
 ('helpful_votes', 'int'),
 ('total_votes', 'int'),
 ('vine', 'string'),
 ('verified_purchase', 'string')]

In [8]:
# 1. Filter the data and create a new DataFrame or table to retrieve all the rows where the total_votes count is equal to or greater than 20
total_votes_df = vine_df.filter("total_votes>=20")
total_votes_df.show()

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
|R1B4753QMTD8ON|          5|           32|         32|   N|                N|
|R28BVHOEL6GCHZ|          5|           21|         21|   N|                Y|
|R27IB4BRKO71I0|          2|           16|         20|   N|                Y|
| RJ86OW12Y4NRL|          5|          124|        128|   N|                Y|
|R3KRABOWCH6ZE4|          1|           48|         65|   N|                Y|
|R1OAKARA30DIWX|          4|           83|         89|   N|                Y|
|R36LSV4WQ4VB1K|          5|           17|         22|   N|                N|
|R18BTOE5UGN7UA|          5|           26|         28|   N|                Y|
|R1DXLGDSK2WMLD|          4|           22|         22|   N|                N|
| RZG2OHQ12YYZI|          5|           30|         31|   N|     

In [10]:
# 2. Filter the new DataFrame or table created in Step 1 and create a new DataFrame or table to retrieve all the rows where
#     the number of helpful_votes divided by total_votes is equal to or greater than 50%.
helpful_votes_df = total_votes_df.filter((total_votes_df.helpful_votes/total_votes_df.total_votes) >= 0.5)
helpful_votes_df.show()

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
|R1B4753QMTD8ON|          5|           32|         32|   N|                N|
|R28BVHOEL6GCHZ|          5|           21|         21|   N|                Y|
|R27IB4BRKO71I0|          2|           16|         20|   N|                Y|
| RJ86OW12Y4NRL|          5|          124|        128|   N|                Y|
|R3KRABOWCH6ZE4|          1|           48|         65|   N|                Y|
|R1OAKARA30DIWX|          4|           83|         89|   N|                Y|
|R36LSV4WQ4VB1K|          5|           17|         22|   N|                N|
|R18BTOE5UGN7UA|          5|           26|         28|   N|                Y|
|R1DXLGDSK2WMLD|          4|           22|         22|   N|                N|
| RZG2OHQ12YYZI|          5|           30|         31|   N|     

In [12]:
# 3. Filter the DataFrame or table created in Step 2, and create a new DataFrame or table that retrieves 
#     all the rows where a review was written as part of the Vine program (paid), vine == 'Y'

paid_vine_df = helpful_votes_df.filter("vine == 'Y'")
paid_vine_df.show()

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
|R32OMS1LSDRCOB|          4|           27|         31|   Y|                N|
|R2IPKK4I0XRCFR|          4|           16|         21|   Y|                N|
| RXYYJ8SGI9IX6|          5|           36|         41|   Y|                N|
|R1SAUBFC7T6JHC|          5|           73|         75|   Y|                N|
|R329HVJKKHWSG3|          5|           59|         59|   Y|                N|
| RZL21K50M52KW|          5|           22|         22|   Y|                N|
|R24RC5J2D92SBE|          5|           19|         20|   Y|                N|
|R1803HITBY7Y08|          4|           92|        104|   Y|                N|
|R1888B8Z6QWYZ4|          4|           18|         25|   Y|                N|
|R1S538NK2I5PRL|          5|           21|         22|   Y|     

In [13]:
# 4. Retrieve all the rows where the review was not part of the Vine program (unpaid), vine == 'N'.
unpaid_vine_df = helpful_votes_df.filter("vine == 'N'")
unpaid_vine_df.show()

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
|R1B4753QMTD8ON|          5|           32|         32|   N|                N|
|R28BVHOEL6GCHZ|          5|           21|         21|   N|                Y|
|R27IB4BRKO71I0|          2|           16|         20|   N|                Y|
| RJ86OW12Y4NRL|          5|          124|        128|   N|                Y|
|R3KRABOWCH6ZE4|          1|           48|         65|   N|                Y|
|R1OAKARA30DIWX|          4|           83|         89|   N|                Y|
|R36LSV4WQ4VB1K|          5|           17|         22|   N|                N|
|R18BTOE5UGN7UA|          5|           26|         28|   N|                Y|
|R1DXLGDSK2WMLD|          4|           22|         22|   N|                N|
| RZG2OHQ12YYZI|          5|           30|         31|   N|     

In [15]:
# 5. Determine the total number of reviews, the number of 5-star reviews, and the percentage of 5-star reviews for the two types of review (paid vs unpaid).
vine_df.count()

4850360

In [16]:
total_votes_df.count()

67855

In [17]:
helpful_votes_df.count()

61948

In [23]:
paid_reviews = paid_vine_df.count()
paid_reviews

334

In [24]:
unpaid_reviews = unpaid_vine_df.count()
unpaid_reviews

61614

In [21]:
paid_5star_reviews = paid_vine_df.filter(paid_vine_df.star_rating==5).count()
paid_5star_reviews

139

In [22]:
unpaid_5star_reviews = unpaid_vine_df.filter(unpaid_vine_df.star_rating==5).count()
unpaid_5star_reviews

32665

In [25]:
paid_5star_percentage = (paid_5star_reviews/paid_reviews) * 100
paid_5star_percentage

41.61676646706587

In [26]:
unpaid_5star_percentage = (unpaid_5star_reviews/unpaid_reviews) * 100
unpaid_5star_percentage

53.01554841432142